In [1]:
import tensorflow as tf
from tensorflow import keras

# Creating a Convolutional Neural Network

### Access images located in a local directory

In [2]:
# the os library gives you  access to the file system
import os

# We will be using images of fruit (apples and oranges),
# so our classification will be binary (apple or orange)


# listdir returns a list of the files in the directory; len() returns the number of files/items in the list
# display the number of images in the various folders

# the directory location is relative to the location of the Notebook

# Training Images
# "fruit/train/apple"
# "fruit/train/orange"

# Test Images
# "fruit/test/apple"
# "fruit/test/orange"

In [3]:
# determine the number of images of each type

# 147 training images
print("The number of train apple images: ", len(os.listdir("fruit/train/apple")))
print("The number of train orange images: ", len(os.listdir("fruit/train/orange")))

#37 test images
print("The number of test apple images: ", len(os.listdir("fruit/test/apple")))
print("The number of test orange images: ", len(os.listdir("fruit/test/orange")))

The number of train apple images:  75
The number of train orange images:  72
The number of test apple images:  19
The number of test orange images:  18


### Create a CNN

In [4]:
# larger input shapes tend to require more convolutions to get them down to size

model = tf.keras.models.Sequential([
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), padding='same', activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a Dense (fully-connected) neural network
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1; 0 is for 'apple' and 1 is for 'orange'
    # a single (binary) output neuron requires a 'sigmoid' activation
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])


# complie model with selected parameters; since the output is binary, 'binary_crossentropy' is used
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
# view the journey of the image through the convolutions
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 37, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 18, 18, 64)        36928     
__________

### Normalize the data and generate labels for the images

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be normalized (rescaled by 1./255)
train_data_generator = ImageDataGenerator(rescale=1/255)
validate_data_generator = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 30 using train_data_generator
train_generator = train_data_generator.flow_from_directory(
        'fruit/train',  # This is the source directory for training images 
                        # It contains sub-directories(whose names will be the labels) that contain images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=30,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validate_generator = validate_data_generator.flow_from_directory(
        'fruit/test',  # This is the source directory for training images 
                            # that contains sub-directories(whose names will be the labels) that contain images
        target_size=(150, 150),  # All images will be resized to 150x1500
        batch_size=10,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

Found 147 images belonging to 2 classes.
Found 37 images belonging to 2 classes.


### Train the CNN (using the generated labels)

In [8]:
# fit the model using the generated labels

history = model.fit_generator(
      train_generator,
      steps_per_epoch=5,  
      epochs=5,
    
      # validate the trained model on unseen (test) images
      validation_data = validate_generator,
      validation_steps=4,
      verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5


C:\Users\r560847\AppData\Local\Continuum\anaconda3\lib\site-packages\PIL\Image.py:952: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  ' expressed in bytes should be converted ' +


5/5 [==============================] - 5s 992ms/step - loss: 0.7078 - acc: 0.5986 - val_loss: 0.6562 - val_acc: 0.4865
Epoch 2/5
5/5 [==============================] - 4s 716ms/step - loss: 0.6109 - acc: 0.7823 - val_loss: 0.4407 - val_acc: 0.9459
Epoch 3/5
5/5 [==============================] - 3s 632ms/step - loss: 0.2671 - acc: 0.9524 - val_loss: 0.2145 - val_acc: 0.8919
Epoch 4/5
5/5 [==============================] - 3s 646ms/step - loss: 0.1321 - acc: 0.9592 - val_loss: 0.0121 - val_acc: 1.0000
Epoch 5/5
5/5 [==============================] - 3s 613ms/step - loss: 0.0800 - acc: 0.9592 - val_loss: 0.0137 - val_acc: 1.0000


# Exercises

Experiment with different models and see how the accuracy results differ. In particular change the following parameters:

Set the input padding to 'valid' from 'same'.

Modify the size that all of the images are initially sized to (this parameter will also need to be set to the same value in the generator).

Add or remove convolutional/pooling layers. 

Experiment with the number of neurons used in the Dense (hidden) layer.

Modify the number of epochs used in model.fit_generator()


### Manual Validation

In [ ]:
import numpy as np
from keras.preprocessing import image

# a list of images (from a directory/folder) that will be used to validate the model's predictions
validating_images = os.listdir("fruit/validating_images")

# predicting the class of unseen images
for file_name in validating_images:
    path = 'fruit/validating_images/' + file_name
    img = image.load_img(path, target_size = (150,150))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)  # the model expects a batch, make a batch of 1
    images = np.vstack([x])
    
    classes = model.predict(images, batch_size = 1)
    print(classes[0])
    if classes[0] >.05:
        print(file_name + " is an orange.")
    else:
        print(file_name + " is an apple.")
    